## Calculate CHIRTS seasonal and annual maximum temperature climatologies over the central Congo Basin
- plots of mean annual and seasonal maximum temperature climatologies
- seasonal distribution plots

In [1]:
import pandas as pd
import xarray as xr
import numpy as np
xr.set_options(cmap_sequential='jet')
import matplotlib.pyplot as plt
import warnings
import netCDF4
import datetime
import plotly
import plotly.express as px
import plotly.graph_objects as go
from affine import Affine
import cartopy.crs as ccrs
#import nco
import xesmf as xe
from calendar import monthrange
import pickle as pkl
import geopandas

In [2]:
# only use this after code is fully working to supress the plot warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
WRKDIR = '/home/s0677837/Datastore/PHD/'
Chirps_clim = WRKDIR + 'CHIRPS/Clim_monthly/'
CHIRTS = WRKDIR + 'CHIRTS/'
PKL = WRKDIR + 'PKL/'

In [4]:
# full map extent (from full vegetation map extent)
lat_min = -5.06319436
lat_max = 3.29741657
lon_min = 14.99769448
lon_max = 25.78052774

In [ ]:
%%time
T_clim = []

for mo in range(1,13):
    mo = str(mo).zfill(2)
    T_yrs = []

    for yr in range(1983,2010):
        # opening each image
        ds = xr.open_rasterio(CHIRTS + 'CHIRTSmax.'+ str(yr) + '.' + mo +'.tif')

        # renaming x and y dimensions to the same as the ALOS imagery
        ds = ds.rename({'x': 'longitude','y': 'latitude', })

        # cutting out to extent of vegetation map
        im = ds.where((lon_min < ds.longitude) & (ds.longitude < lon_max) & (lat_min < ds.latitude) & (ds.latitude < lat_max), drop=True)            

        # setting no values (currently -9999.0) to nan
        im = im.where(im != -9999., drop=True)   

        # converting from DataArray to Dataset format
        im = im.to_dataset(dim='band')

        # creating values and key pairs to rename the variables within the final concatenated array 
        # they are currently all labelled 1
        values = []
        values.append(str(yr))
        keys = im.keys()
        dictionary = dict(zip(keys, values))

        # changing the variable name to the month value
        im = im.rename_vars(dictionary)
        df = im.to_dataframe().reset_index()
        df['year'] = str(yr)
        df.columns = ['latitude','longitude','value','year']
        T_yrs.append(df)

    # appending each yearly image to the array of all images
    T_yrs = pd.concat(T_yrs)
    T_yrs['month'] = mo
    T_clim.append(T_yrs)

T_clim = pd.concat(T_clim)


In [ ]:
T_clim

In [ ]:
# now calculating the monthly climatology
T_clim_m = T_clim.groupby(['latitude','longitude','month']).mean().reset_index()

In [ ]:
# calculating seasonal climatologies
T_clim_DJF = T_clim_m[T_clim_m['month'].isin(['12','01','02'])].groupby(['latitude','longitude']).mean().reset_index()
T_clim_MAM = T_clim_m[T_clim_m['month'].isin(['03','04','05'])].groupby(['latitude','longitude']).mean().reset_index()
T_clim_JJA = T_clim_m[T_clim_m['month'].isin(['06','07','08'])].groupby(['latitude','longitude']).mean().reset_index()
T_clim_SON = T_clim_m[T_clim_m['month'].isin(['09','10','11'])].groupby(['latitude','longitude']).mean().reset_index()


In [ ]:
# naming the array columns
T_clim_DJF.columns = ['latitude','longitude','DJF']
T_clim_MAM.columns = ['latitude','longitude','MAM']
T_clim_JJA.columns = ['latitude','longitude','JJA']
T_clim_SON.columns = ['latitude','longitude','SON']

In [ ]:
# creating an xarray including all four seasons
DJF = T_clim_DJF.set_index(['latitude','longitude']).to_xarray()
MAM = T_clim_MAM.set_index(['latitude','longitude']).to_xarray()
JJA = T_clim_JJA.set_index(['latitude','longitude']).to_xarray()
SON = T_clim_SON.set_index(['latitude','longitude']).to_xarray()

clim_chirts_seasons = xr.merge([DJF,MAM,JJA,SON])

print(clim_chirts_seasons)

In [ ]:
# Saving to pkl
with open(PKL + 'clim_chirts_seasons_DJF.pkl', 'wb') as f:
    pkl.dump(T_clim_DJF, f)
    
with open(PKL + 'clim_chirts_seasons_MAM.pkl', 'wb') as f:
    pkl.dump(T_clim_MAM, f)

with open(PKL + 'clim_chirts_seasons_JJA.pkl', 'wb') as f:
    pkl.dump(T_clim_JJA, f)

with open(PKL + 'clim_chirts_seasons_SON.pkl', 'wb') as f:
    pkl.dump(T_clim_SON, f)
    
with open(PKL + 'clim_chirts_seasons.pkl', 'wb') as f:
    pkl.dump(clim_chirts_seasons, f)    

In [ ]:
clim_chirts_seasons = pkl.load(open(PKL + 'clim_chirts_seasons.pkl', 'rb'))

In [ ]:
df = clim_chirts_seasons.to_dataframe().reset_index()

In [ ]:
df[['DJF','MAM','JJA','SON']].mean(axis=1)

### Calculating mean annual maximum temperature

In [ ]:
df['Mean annual temperature'] = df[['DJF','MAM','JJA','SON']].mean(axis=1)

In [ ]:
df2=df[['latitude','longitude','Mean annual temperature']]
ds2 = df2.set_index(['latitude','longitude']).to_xarray()
ds2

In [ ]:
# reading in the shapefile footprint
import shapefile as shp
footprint = shp.Reader(WRKDIR + '/Veg_maps/CC_footprint.shp')

In [ ]:
import seaborn as sns

# lake and river features to be added to maps
import cartopy.crs as ccrs
import cartopy.feature
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

rivers = cartopy.feature.NaturalEarthFeature(
    category='physical', name='rivers_lake_centerlines',
    scale='10m', facecolor='none', edgecolor='black')

lakes = cartopy.feature.NaturalEarthFeature(
    category='physical', name='lakes',
    scale='10m', facecolor='none', edgecolor='aqua')

def plot_map(ds,variable,cmap_name,label,t,b,l,r,vmin,vmax):
    # inputs: data array, variable, colour map, legend lable, plot ticks for top, bottom, left and right,cbar min, cbar max
    # e.g. plot_map(clim_chirps_annual,'Annual','viridis_r','Annual rainfall (mm)',True,True,True,True,0,2100)
    sns.set(font_scale=1.7, style="white")

    fig, ax = plt.subplots(
        nrows=1, ncols=1, subplot_kw={'projection': ccrs.PlateCarree()},
                             figsize=(11.1, 9.4))
    ax.add_feature(rivers, linewidth=1)
    ax.add_feature(lakes, linewidth=1)

    lat_min = -5.03769436
    lat_max = 3.26980564
    ax.set_extent([15,23,-4, lat_max])
    cmap = plt.cm.get_cmap(cmap_name)
    im = ds[variable].plot(cmap=cmap,vmin=vmin,vmax=vmax,cbar_kwargs=dict(orientation='horizontal', pad=0.07,shrink=0.71,label=label))
    # adding 2nd axes labels
    ax.tick_params(labeltop=True, labelright=True)

    for shape in footprint.shapeRecords():
        x = [i[0] for i in shape.shape.points[:]]
        y = [i[1] for i in shape.shape.points[:]]
    plt.plot(x,y, color='grey')
    
    labelsx=['0','16$^\circ$E','18$^\circ$E','20$^\circ$E','22$^\circ$E']
    ax.set_xticklabels(labelsx)
    ax.set_xlim(15,23)
    # turning off x and y label titles
    x_axis = ax.axes.get_xaxis()
    x_label = x_axis.get_label()
    x_label.set_visible(False)

    y_axis = ax.axes.get_yaxis()
    y_label = y_axis.get_label()
    y_label.set_visible(False)


    labelsy=['0','4$^\circ$S','3$^\circ$S','2$^\circ$S','1$^\circ$S','0$^\circ$','1$^\circ$N','2$^\circ$N','3$^\circ$N']
    ax.set_yticklabels(labelsy)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=1, color='gray', alpha=0.5, linestyle='--')
    gl.xlabels_top = t
    gl.xlabels_bottom = b
    gl.ylabels_left = l
    gl.ylabels_right = r
    gl.xlines = True

    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlocator = mticker.FixedLocator([16, 17, 18, 19, 20, 21, 22, 23])
    gl.ylocator = mticker.FixedLocator([-4,-3,-2,-1,0,1,2,3])
    gl.ylabel_style = {'size': 15, 'color': 'black'}
    gl.xlabel_style = {'size': 15, 'color': 'black'}


    plt.show()    

In [ ]:
def plot_map_seasons(ds,variable,cmap_name,label,t,b,l,r,vmin,vmax):
    # inputs: data array, variable, colour map, legend lable, plot ticks for top, bottom, left and right,cbar min, cbar max
    # e.g. plot_map(clim_chirps_annual,'Annual','viridis_r','Annual rainfall (mm)',True,True,True,True,0,2100)
    sns.set(font_scale=1.7, style="white")

    fig, ax = plt.subplots(
        nrows=1, ncols=1, subplot_kw={'projection': ccrs.PlateCarree()},
                             figsize=(11.1*1.1, 9.4*1.1))
    ax.add_feature(rivers, linewidth=1)
    ax.add_feature(lakes, linewidth=1)

    lat_min = -5.03769436
    lat_max = 3.26980564
    ax.set_extent([15,23,-4, lat_max])
    cmap = plt.cm.get_cmap(cmap_name)
    im = ds[variable].plot(cmap=cmap,vmin=vmin,vmax=vmax,cbar_kwargs=dict(orientation='horizontal', pad=0.07,shrink=0.71,label=label))
    # adding 2nd axes labels
    ax.tick_params(labeltop=True, labelright=True)

    labelsx=['0','16$^\circ$E','18$^\circ$E','20$^\circ$E','22$^\circ$E']
    ax.set_xticklabels(labelsx)
    ax.set_xlim(15,23)
    # turning off x and y label titles
    x_axis = ax.axes.get_xaxis()
    x_label = x_axis.get_label()
    x_label.set_visible(False)

    y_axis = ax.axes.get_yaxis()
    y_label = y_axis.get_label()
    y_label.set_visible(False)


    labelsy=['0','4$^\circ$S','3$^\circ$S','2$^\circ$S','1$^\circ$S','0$^\circ$','1$^\circ$N','2$^\circ$N','3$^\circ$N']
    ax.set_yticklabels(labelsy)

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                      linewidth=1, color='gray', alpha=0.5, linestyle='--')
    gl.xlabels_top = t
    gl.xlabels_bottom = b
    gl.ylabels_left = l
    gl.ylabels_right = r
    gl.xlines = True

    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlocator = mticker.FixedLocator([16, 17, 18, 19, 20, 21, 22, 23])
    gl.ylocator = mticker.FixedLocator([-4,-3,-2,-1,0,1,2,3])
    gl.ylabel_style = {'size': 17, 'color': 'black'}
    gl.xlabel_style = {'size': 17, 'color': 'black'}


    plt.show()

In [ ]:
# plotting mean annual maximum temperature
plot_map(ds2,'Mean annual temperature','jet','Mean annual max temperature (\xb0C) ',True,True,True,True,27,33)

In [ ]:
plot_map_seasons(clim_chirts_seasons,'DJF','jet','Mean seasonal max temperature (\xb0C) ',True,False,True,False,27,33)
plot_map_seasons(clim_chirts_seasons,'MAM','jet','Mean seasonal max temperature (\xb0C) ',True,False,False,True,27,33)
plot_map_seasons(clim_chirts_seasons,'JJA','jet','Mean seasonal max temperature (\xb0C) ',False,True,True,False,27,33)
plot_map_seasons(clim_chirts_seasons,'SON','jet','Mean seasonal max temperature (\xb0C) ',False,True,False,True,27,33)

In [ ]:
clim_chirts_seasons

In [ ]:
# plotting seasonal temperature in a grid
da = clim_chirts_seasons.to_array()

import matplotlib as mpl
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16

ax = da.plot(x='longitude',y='latitude',col='variable',col_wrap=2,cmap='jet', robust=True, cbar_kwargs={"label": "Seasonal temperature climatology (\xb0C)"},)
ax.set_ticks(fontsize='xx-small')
ax.set_titles(fontsize='small')
ax.set_xlabels(fontsize='x-small')
ax.set_ylabels(fontsize='x-small')


### Plotting the disribution of seasonal maximum temperatures across the Congo Basin

In [ ]:
subplot = T_clim_DJF['DJF'].hist(bins=1000,edgecolor='none')
subplot.set_xlim((28,33))

In [ ]:
subplot = T_clim_MAM['MAM'].hist(bins=1000,edgecolor='none')
subplot.set_xlim((28,33))

In [ ]:
subplot = T_clim_JJA['JJA'].hist(bins=1000,edgecolor='none')
subplot.set_xlim((28,33))

In [ ]:
subplot = T_clim_SON['SON'].hist(bins=1000,edgecolor='none')
subplot.set_xlim((28,33))